In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/babble/data/')
# os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_spouse'
os.environ['SNORKELDB'] = 'postgres://localhost:5432/babble_test_bike'


from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

# Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
# candidate_class = Spouse

Biker = candidate_subclass('Biker', ['person', 'bike'])
candidate_class = Biker

In [3]:
candidates = session.query(candidate_class).filter(candidate_class.split == 1).all()
print(len(candidates))

1037


In [4]:
import os
from mturk_processing import MTurkHelper
helper = MTurkHelper(candidates, num_hits=25)

In [5]:
output_csv_path= os.environ['SNORKELHOME'] + '/tutorials/babble/bike/data/mturk_visual_out.csv'
explanations = helper.postprocess_visual(output_csv_path, verbose=False)

Num HITs unique: 25
Num HITs total: 75
Unanimous: 65
Majority: 24
Bad: 11


In [14]:
for exp in explanations:
    if not isinstance(exp.candidate, basestring):
        print('\t' + exp.candidate.get_stable_id())
    else:
        print(exp.candidate)

1:140::bbox:5~~1:140::bbox:11
1:140::bbox:5~~1:140::bbox:11
1:140::bbox:5~~1:140::bbox:11
1:50::bbox:1~~1:50::bbox:6
1:50::bbox:1~~1:50::bbox:6
1:50::bbox:1~~1:50::bbox:6
1:61::bbox:15~~1:61::bbox:13
1:61::bbox:15~~1:61::bbox:13
1:61::bbox:15~~1:61::bbox:13
1:12::bbox:10~~1:12::bbox:1
1:12::bbox:10~~1:12::bbox:1
1:12::bbox:10~~1:12::bbox:1
1:74::bbox:5~~1:74::bbox:26
1:74::bbox:5~~1:74::bbox:26
1:74::bbox:5~~1:74::bbox:26
1:75::bbox:8~~1:75::bbox:7
1:75::bbox:8~~1:75::bbox:7
1:75::bbox:8~~1:75::bbox:7
1:197::bbox:11~~1:197::bbox:1
1:197::bbox:11~~1:197::bbox:1
1:197::bbox:11~~1:197::bbox:1
	1:71::bbox:2~~1:71::bbox:0
	1:71::bbox:2~~1:71::bbox:0
1:208::bbox:6~~1:208::bbox:0
1:208::bbox:6~~1:208::bbox:0
1:116::bbox:2~~1:116::bbox:1
1:116::bbox:2~~1:116::bbox:1
1:116::bbox:2~~1:116::bbox:1
1:3::bbox:14~~1:3::bbox:15
1:3::bbox:14~~1:3::bbox:15
1:3::bbox:14~~1:3::bbox:15
1:166::bbox:1~~1:166::bbox:4
1:166::bbox:1~~1:166::bbox:4
	1:80::bbox:1~~1:80::bbox:0
	1:80::bbox:1~~1:80::bbox:0
1:39::b

In [10]:
from pprint import pprint
pprint(sorted(c.get_stable_id() for c in candidates))

[u'1:0::bbox:16~~1:0::bbox:0',
 u'1:0::bbox:16~~1:0::bbox:1',
 u'1:0::bbox:16~~1:0::bbox:2',
 u'1:0::bbox:3~~1:0::bbox:1',
 u'1:100::bbox:0~~1:100::bbox:5',
 u'1:100::bbox:1~~1:100::bbox:3',
 u'1:101::bbox:2~~1:101::bbox:0',
 u'1:102::bbox:1~~1:102::bbox:0',
 u'1:102::bbox:3~~1:102::bbox:0',
 u'1:103::bbox:11~~1:103::bbox:1',
 u'1:103::bbox:11~~1:103::bbox:10',
 u'1:103::bbox:2~~1:103::bbox:1',
 u'1:103::bbox:2~~1:103::bbox:10',
 u'1:103::bbox:2~~1:103::bbox:12',
 u'1:103::bbox:2~~1:103::bbox:14',
 u'1:103::bbox:3~~1:103::bbox:0',
 u'1:103::bbox:4~~1:103::bbox:0',
 u'1:103::bbox:4~~1:103::bbox:13',
 u'1:103::bbox:5~~1:103::bbox:14',
 u'1:103::bbox:6~~1:103::bbox:13',
 u'1:103::bbox:7~~1:103::bbox:0',
 u'1:103::bbox:7~~1:103::bbox:13',
 u'1:103::bbox:8~~1:103::bbox:1',
 u'1:103::bbox:8~~1:103::bbox:10',
 u'1:103::bbox:9~~1:103::bbox:14',
 u'1:105::bbox:14~~1:105::bbox:0',
 u'1:105::bbox:4~~1:105::bbox:0',
 u'1:105::bbox:5~~1:105::bbox:0',
 u'1:106::bbox:0~~1:106::bbox:11',
 u'1:106::bbo

In [ ]:
# exp_iterator = iter(explanations)

In [ ]:
# exp = exp_iterator.next()
# from snorkel.viewer import SentenceNgramViewer
# print(exp.condition)
# sv = SentenceNgramViewer([exp.candidate], session, n_per_page=3, height=150)
# sv

In [ ]:
for exp in explanations[:5]: print(exp.condition)

In [ ]:
# from snorkel.contrib.babble import Explanation
# exp = Explanation("person1 equals person2", True, name='test_exp')
# homemades = [exp]

In [ ]:
from snorkel.contrib.babble import Babbler
user_lists = {}
babbler = Babbler(mode='image', 
                  candidate_class=candidate_class, 
                  explanations=explanations)
# babbler.apply(parallelism=1)

In [ ]:
lfs = babbler.generate_lfs()

In [ ]:
babbler.filter_duplicate_semantics()

In [ ]:
babbler.filter_consistency()

In [ ]:
from snorkel.contrib.babble import sem_to_str

for parse in babbler.parses:
    print(sem_to_str(parse.semantics))

In [ ]:
babbler.get_explanations()

In [ ]:
%time babbler.generate_label_matrix(split=1, parallelism=1)

In [ ]:
# babbler.load_matrix(session, split=1)

In [ ]:
babbler.filter_uniform_signatures()

In [ ]:
babbler.filter_duplicate_signatures()

In [ ]:
for p in babbler.parses: print(sem_to_str(p.semantics))

In [ ]:
babbler.get_explanations()

In [ ]:
babbler.label_matrix.lf_stats(session, labels=L_gold_dev)